# Game Playing Agents #

In this project, we will implement a variation of **Minimax** to play a simple strategic game.

The game is played on a 2D grid on which there are a number of enemy bases. The agent must destroy the enemy bases to win the game. Here is an example of a 5x5 map:

```
[] T3 [] B1 []
[] [] T2 [] []
A  [] [] [] []
[] T1 [] T0 []
[] [] T4 [] B0
```
The agent is specified by `A`. There are two bases, `B0` and `B1`. There are five towers: `T0` through `T4`.

Towers are and bases are automatically destroyed when the agent encounters them and they are no longer invincible. But when the agent encounters invincible towers and bases, the agent is automatically killed. This game emulates the high-level strategic planning, wherein an encounter between the agent and a tower or base might represent a drawn-out firefight.

However, the towers must be destroyed in a specific order. The bases are ``guarded`` by towers. This means bases are invulnerable until the guarding towers are destroyed. Some towers may also be ``guarded`` by other towers and thus be invulnerable until the guarding towers are destroyed.

For example, in the above map it might be the case that:
```
B0: guarded by T0
B1: guarded by T1
T0: guarded by T3 and T4
T1: guarded by T2
```
The game can be won by destroying towers in the following order: T2, T1, B1, T3, T4, T0, B0.
There are many possible orderings and the agent must discover one of the correct orderings.

The guard configuration is represented internally in the game engine with a dictionary that you will be able to see but the agent will not be able to see:
```
{'B0': ['T0'], 'B1': ['T1'], 'T1': ['T2'], 'T0': ['T3', 'T4']}
```
The agent must learn one of the correct orderings through trial and error using a **Minimax**-like algorithm .

To facilitate this, we provide a means of scoring the agent's attempts. The agent gets -100 points for dying, either from encountering a tower or base that is still guarded, or by going off the edge of the map.
The agent gets +100 points for each tower destroyed.
The agent gets +200 points for each based destroyed.
The game ends when the agent is killed or when all bases are destroyed.

The project comes is a set number of hand-crafted maps, but is also capable of generating new maps of arbitrary complexity.

**Submission instructions:** Complete the ```maximax``` function below. Run the notebook on the test cases at the bottom of the notebook. Submit your notebook *with cell output saved*. The notebook will be graded by looking at the traces of your outputs to see how many of the test cases were successfully won by the agent.

You can only modify the ```maximax``` cell. You can make additional helper function and make additional code and text cells to complete your implementation. You cannot modify any of the other code cells. In the tests, you can change the ```max_depth``` parameter as necessary.

Please disable any debugging print statements that you added for yourself.

**Grading:** One point per game won (6 games total). Grades will be determine by manual inspection of your notebook game outputs. No points will be given if you implement an algorithm other than maximax (e.g. A* or a rule-based agent), as determined by visual inspection. Visual inspection will not check for algorithm correctness, which is determined by your agent's ability to play the games.



# Install and import

In [18]:
!pip install graphviz
!pip install pydot

In [19]:
from typing import NamedTuple
from copy import copy, deepcopy
from enum import Enum
from functools import partial
import random
import math
import graphviz
import pydot
from IPython.display import Image

# Data Structures

An `Entity` can be the agent, tower, or base. The entity's status is one of the enumerated types below.

A `State` object contains all information relevant to the state of the game. Note that `guards` slot indicates which entities are *guarded by* which other agents. That is, `{B0: [T1, T2]}` indicates that base `B0` is guarded by towers `T1` and `T2`, meaning `B0` cannot be destroyed until `T1` and `T2` are destroyed.

In [20]:
### Entity class: can be agents, towers, or bases
class Entity():
  def __init__(self, name, x, y, status):
    self.name = name          # Name
    self.x = x                # x position
    self.y = y                # y position
    self.status = status      # status from Status ENUM

### State object
class State():
  def __init__(self, agent, towers = [], bases = [], terminal = False, rows=3, cols=3, guards = {}):
    self.agent = agent            # pointer to an agent Entity
    self.towers = towers          # list of tower Entity objects
    self.bases = bases            # list of base Entity objects
    self.terminal = terminal      # Is the state terminal (boolean)
    self.rows = rows              # Number of rows in grid
    self.cols = cols              # number of columns in grid
    self.guards = guards          # A dictionary of entity names (string) where
                                  # values are lists of entity names
                                  # E.g. {B0: [T1, T2]} means base B0 is guarded by towers T1 and T2
                                  # and B0 will only be vulnerable when T1 and T2 are Status.DEAD.

#Agent Actions

These functions describe the actions the agent can perform.

In [21]:
def agent_east(state):
  state.agent.x = state.agent.x + 1
  return state

def agent_north(state):
  state.agent.y = state.agent.y - 1
  return state

def agent_south(state):
  state.agent.y = state.agent.y + 1
  return state

def agent_west(state):
  state.agent.x = state.agent.x - 1
  return state

# Enumerated Types

In [22]:
### Types of entities
class EntityTypes(Enum):
  AGENT = 'A'
  BASE = 'B'
  TOWER = 'T'

### Agents, towers, and bases can be alive or dead
class Status(Enum):
  ALIVE = 1
  DEAD = 0

### List of available actions
### Values are functions.
### e.g., Call Actions.EAST.value(state)
class Actions(Enum):
  EAST = partial(agent_east)
  SOUTH = partial(agent_south)
  NORTH = partial(agent_north)
  WEST = partial(agent_west)

# Drawing the Game State

A grid is a list of lists.

In [23]:
### Make a grid as a list of lists

def make_grid(cols, rows):
  grid = [[0 for col in range(cols)] for row in range(rows)]
  return grid

### Put names of agents, towers, and bases on a grid (when alive)

def draw_state(state):
  grid = make_grid(state.cols, state.rows)
  for tower in state.towers:
    if tower.status == Status.ALIVE and tower.x >= 0 and tower.x < state.cols and tower.y >=0 and tower.y < state.rows:
      grid[tower.y][tower.x] = tower.name
  for base in state.bases:
    if base.status == Status.ALIVE and base.x >= 0 and base.x < state.cols and base.y >=0 and base.y < state.rows:
      grid[base.y][base.x] = base.name
  if state.agent.x >= 0 and state.agent.x < state.cols and state.agent.y >=0 and state.agent.y < state.rows:
    grid[state.agent.y][state.agent.x] = EntityTypes.AGENT.value
  return grid

### Make game state easy to view
### indent: shift everything to the right by a number of spaces

def pretty_print(state, indent = 0):
  # Make a grid with agent, towers, and bases
  grid = draw_state(state)
  # Now redraw the grid line by line
  for j in range(len(grid)):
    line = ''
    for i in range(len(grid[j])):
      c = grid[j][i]
      if c == 0:
        c = '[]'
      if len(c) < 2:
        c = c + " "
      line = line + c + " "
    print(' '*indent + line)
  print(' '*indent + "  guards", state.guards)
  print(' '*indent + "  agent", state.agent.status.name)
  print(' '*indent + "  towers", [tower.name + ": " + str(tower.status.name) for tower in state.towers])
  print(' '*indent + "  bases", [base.name + ": " + str(base.status.name) for base in state.bases])
  print(' '*indent + "  terminal", state.terminal)

# Make a Random Game

This is optional code provided if you want to generate new maps to test your agent on.

In [24]:
### Make the towers, bases, and their dependencies (who defends who).
### Each base is always defended by at least one tower
### num_towers must be equal to or greater than num_bases

def make_towers_and_bases(num_bases = 1, num_towers = 1):
  # Make bases
  bases = [str(EntityTypes.BASE.value)+str(b) for b in list(range(num_bases))]
  # Make towers
  towers = [str(EntityTypes.TOWER.value)+str(b) for b in list(range(num_towers))]
  # Initialize defended hash
  guards = {}
  # initially each base should have a tower
  for i, base in enumerate(bases):
    guards[base] = [towers[i]]
  # Keep track of used bases and towers. All bases are used up and some towers equal to number of bases
  used = bases + towers[:i+1]
  # Keep track of remaining towers
  remaining_towers = towers[i+1:]
  # Iterate through remaining towers, assigning them to defend
  while len(remaining_towers) > 0:
    next_tower = remaining_towers.pop(0)
    pick = used[random.randrange(0, len(used))]
    if pick not in guards:
      guards[pick] = []
    guards[pick].append(next_tower)
    used.append(pick)
  return bases, towers, guards

In [25]:
### Make towers and bases then assign them to grid locations
### Make an initial state object

def make_random_game(num_bases = 1, num_towers = 1):
  # Make bases and towers and assign defenders
  bases, towers, guards = make_towers_and_bases(num_bases, num_towers)
  # Everything altogether
  all = bases + towers
  # We will always have 5 rows
  # Compute how many columns we need
  num_cols = int(math.ceil(len(all)/4.0)) + 3
  # Make a set of valid positions
  positions = []
  # Iterate through all the rows and columsn
  # j loop is for rows
  for j in range(4):
    adjust = 0 # Sometimes we need fewer slots on odd rows
    if num_cols % 2 == 0:
      adjust = 1
    #i loop is for columns
    for i in range(int(num_cols/2) - ((j%2) * adjust)):
      # A column position
      col = (i * 2) + 1
      # If we are on odd rows, add an offset
      if (j % 2) == 1:
        col = col + 1
      # Compute the row number
      row = j
      # Skip the middle row
      if row > 1:
        row = row + 1
      # Store the valid position
      positions.append((col, row))
  # For storing entities when we make them
  tower_entities = []
  base_entities = []
  # Iterate through towers and make entities
  for tower in towers:
    # Grab a valid position
    pos = positions.pop(random.randrange(0, len(positions)))
    # Make a tower entity
    tower_entities.append(Entity(name = tower, x = pos[0], y = pos[1], status = Status.ALIVE))
  # Iterate through bases and make entities
  for base in bases:
    # Grab a valid position
    pos = positions.pop(random.randrange(0, len(positions)))
    # Make a base entity
    base_entities.append(Entity(name = base, x = pos[0], y = pos[1], status = Status.ALIVE))
  # Build the state object
  state = State(agent = Entity(name = "agent", x = 0, y = 2, status = Status.ALIVE),
                towers = [tower for tower in tower_entities],
                bases = [base for base in base_entities],
                terminal = False,
                rows = 5,
                cols = num_cols,
                guards = guards)
  return state

In [26]:
random_game_state = make_random_game(2, 3)
pretty_print(random_game_state)

[] T2 [] B0 [] 
[] [] [] [] [] 
A  [] [] [] [] 
[] B1 [] T1 [] 
[] [] T0 [] [] 
  guards {'B0': ['T0', 'T2'], 'B1': ['T1']}
  agent ALIVE
  towers ['T0: ALIVE', 'T1: ALIVE', 'T2: ALIVE']
  bases ['B0: ALIVE', 'B1: ALIVE']
  terminal False


# Hard-coded initial states

In [27]:
init_state3 = State(agent=Entity(name="agent",x=0,y=1,status=Status.ALIVE),
                 towers=[Entity(name="t0", x=2, y=0,status=Status.ALIVE)],
                 bases=[Entity(name="b0", x=4, y=2,status=Status.ALIVE)],
                 terminal = False,
                 rows = 3,
                 cols = 5,
                 guards = {'b0': ['t0']})

init_state1 = State(agent=Entity(name="agent",x=0,y=1,status=Status.ALIVE),
                 towers=[Entity(name="t0",x=1, y=0,status=Status.ALIVE)],
                 bases=[Entity(name="b0",x=2, y=2,status=Status.ALIVE)],
                 terminal = False,
                 rows = 3,
                 cols = 3,
                 guards = {'b0': ['t0']})

init_state2 = State(agent=Entity(name="agent",x=0,y=1,status=Status.ALIVE),
                     towers=[Entity(name="t0",x=1,y=0,status=Status.ALIVE),
                             Entity(name="t1",x=1,y=2,status=Status.ALIVE)],
                     bases=[Entity(name="b0",x=2, y=1,status=Status.ALIVE)],
                     terminal = False,
                     rows = 3,
                     cols = 3,
                     guards = {'b0': ['t0', 't1']})

init_state4 = State(agent=Entity(name="agent", x=0, y=1, status = Status.ALIVE),
                  towers = [Entity(name="T0",x=3,y=0,status=Status.ALIVE),
                            Entity(name="T1",x=2,y=2,status=Status.ALIVE),
                            Entity(name="T2",x=2,y=0,status=Status.ALIVE),
                            Entity(name="T3",x=3,y=2,status=Status.ALIVE)],
                  bases = [Entity(name="B0",x=4, y=0,status=Status.ALIVE),
                           Entity(name="B1",x=1, y=2,status=Status.ALIVE)],
                  terminal = False,
                  rows = 3,
                  cols = 5,
                  guards = {'B0': ['T0'], 'B1': ['T1'], 'T0': ['T2'], 'T1': ['T3']})

init_state6 = State(agent=Entity(name="agent", x=0, y=0, status = Status.ALIVE),
                   towers = [Entity(name="T0",x=3,y=6,status=Status.ALIVE),
                             Entity(name="T1",x=6,y=6,status=Status.ALIVE),
                             ],
                   bases = [Entity(name="B0",x=0, y=3,status=Status.ALIVE),
                            ],
                   terminal = False,
                   rows = 7,
                   cols = 7,
                   guards = {'B0': ['T0'], 'T0': ['T1']})

init_state5 = State(agent=Entity(name="agent", x=0, y=2, status = Status.ALIVE),
                      towers = [Entity(name="T3",x=1,y=0,status=Status.ALIVE),
                                Entity(name="T2",x=2,y=1,status=Status.ALIVE),
                                Entity(name="T1",x=1,y=3,status=Status.ALIVE),
                                Entity(name="T0",x=3,y=3,status=Status.ALIVE),
                                Entity(name="T4",x=2,y=4,status=Status.ALIVE)
                                ],
                  bases = [Entity(name="B0",x=4, y=4,status=Status.ALIVE),
                           Entity(name="B1",x=3, y=0,status=Status.ALIVE)
                           ],
                  terminal = False,
                  rows = 5,
                  cols = 5,
                  guards = {'B0': ['T0'], 'B1': ['T1'], 'T1': ['T2'], 'T0': ['T3', 'T4']})

# Initialize game state

In [28]:
init_state = deepcopy(init_state1)
pretty_print(init_state)

[] t0 [] 
A  [] [] 
[] [] b0 
  guards {'b0': ['t0']}
  agent ALIVE
  towers ['t0: ALIVE']
  bases ['b0: ALIVE']
  terminal False


# Compute the game score

**Note:** If you call the `score()` function, you must pass in the number of steps the agent has taken, which is the depth of the search.

In [29]:
### +200 for each dead base
### +100 for each dead tower
### -100 for dead agent
### -1 for each depth (length of solution)

def score(state, depth = 0):
  score = 0
  # Is the agent dead?
  if state.agent.status == Status.DEAD:
    score = score - 100
#  else:
  # Any dead towers?
  for tower in state.towers:
    if tower.status == Status.DEAD:
      score = score + 100
  # Any dead bases?
  for base in state.bases:
    if base.status == Status.DEAD:
      score = score + 200
# Length penalty
  score = score - depth
  return score

# Executing Actions

This section contains the logic for the rules of the game--what happens when different actions are executed under different circumstances causing the agent or towers or bases to die. The `do_action()` is the core function of the game.

Helpers:

- `is_alive(entity, state)`: return true or false depending on whether the given entity is alive in the game state.
- `guarded(entity, state)`: return true if the given entity has living guards.

In [30]:
### Figure out if an entity (string name of a base or tower) is alive or not

def is_alive(entity, state):
  # Check to see if entity is the agent
  if state.agent.name == entity:
    return state.agent.name.status == Status.ALIVE
  # Check to see if entity is a tower
  for tower in state.towers:
    if tower.name == entity:
      return tower.status == Status.ALIVE
  # Check to see if entity is a base
  for base in state.bases:
    if base.name == entity:
      return base.status == Status.ALIVE
  return False

### Figure out if a tower or base (name string) is defended.
### An entity is not defended if every defending entity is dead.

def guarded(entity, state):
  if entity in state.guards:
    for guard in state.guards[entity]:
      if is_alive(guard, state):
        return True
  return False

The `do_action()` function. This implements the main action logic of the game.

In [31]:
### Execute an action
### Action is from Actions ENUM.

def do_action(action, state):
  # Make a deep copy of the state
  new_state = deepcopy(state)
  # Call the action function
  new_state = action.value(new_state)
  # Grab a reference to the agent for convenience
  agent = new_state.agent
  # If agent has gone off the board, then it is dead
  if agent.x < 0 or agent.y < 0 or agent.x >= state.cols or agent.y >= state.rows:
    agent.status = Status.DEAD
    new_state.terminal = True
    #new_state = deepcopy(state) # don't die when going off the edge, just don't move
  else:
    # Did the agent encounter a tower?
    for tower in new_state.towers:
      if tower.status == Status.ALIVE and agent.x == tower.x and agent.y == tower.y:
        # Is the tower defended?
        if guarded(tower.name, state):
          # Tower is defended so agent is dead
          agent.status = Status.DEAD
          new_state.terminal = True
        else:
          # Tower was not defended so tower is dead
          tower.status = Status.DEAD
    # Did the agent encounter a base?
    for base in new_state.bases:
      if base.status == Status.ALIVE and agent.x == base.x and agent.y == base.y:
        # Is the base defended?
        if guarded(base.name, state):
          # Base is defended so agent is dead
          agent.status = Status.DEAD
          new_state.terminal = True
        else:
          # Base was not defended so base is dead
          base.status = Status.DEAD
    # Check for alive bases
    alive_bases = 0
    for base in new_state.bases:
      if base.status == Status.ALIVE:
        alive_bases = alive_bases + 1
    # If there are no alive bases, then the game is over
    if alive_bases == 0:
      new_state.terminal = True
  return new_state

Utility function for driving the agent manually. You can give a list of actions (e.g., `[Actions.EAST, Actions.NORTH, etc.]`) and see how the game state changes.

In [32]:
### Utility for driving the agent manually.
### Route is a list of actions from Actions ENUM (e.g., [Actions.EAST, Actions.NORTH, etc.])
def test_run(state, route):
  # Print the game state
  pretty_print(state)
  # Execute each action in route
  for action in route:
    print("\nACTION", action.name, '\n')
    state = do_action(action, state)
    # Print the game state
    pretty_print(state)
  # Print the score
  print("Score:", score(state))
  return state

In [33]:
dummy_state = test_run(init_state, [Actions.EAST, Actions.NORTH, Actions.EAST])

[] t0 [] 
A  [] [] 
[] [] b0 
  guards {'b0': ['t0']}
  agent ALIVE
  towers ['t0: ALIVE']
  bases ['b0: ALIVE']
  terminal False

ACTION EAST 

[] t0 [] 
[] A  [] 
[] [] b0 
  guards {'b0': ['t0']}
  agent ALIVE
  towers ['t0: ALIVE']
  bases ['b0: ALIVE']
  terminal False

ACTION NORTH 

[] A  [] 
[] [] [] 
[] [] b0 
  guards {'b0': ['t0']}
  agent ALIVE
  towers ['t0: DEAD']
  bases ['b0: ALIVE']
  terminal False

ACTION EAST 

[] [] A  
[] [] [] 
[] [] b0 
  guards {'b0': ['t0']}
  agent ALIVE
  towers ['t0: DEAD']
  bases ['b0: ALIVE']
  terminal False
Score: 100


# Maximax

Complete the ```maximax``` function. You may make any number of helper functions. You may make new code and text cells. You may not change the signature of the ```maximax``` function call.

In [34]:
### Maximax function
### state is initial state
### actions is the set of possible actions
### max_depth is how deep the search can go before backtracking

def maximax(state, actions = [act for act in Actions], max_depth = float("inf")):
  max_depth = 20
  best_action = None # Return the best action
  best_action_value = -float("inf") # Return the value of the best action (dummy default is -infinity)
  ### YOUR CODE GOES BELOW
  memo = {}

  def maximax_helper(state, depth, max_depth):
      state_key = (tuple((tower.name, tower.x, tower.y, tower.status) for tower in state.towers),
                  tuple((base.name, base.x, base.y, base.status) for base in state.bases),
                  (state.agent.name, state.agent.x, state.agent.y, state.agent.status),
                  state.terminal, depth)

      if state_key in memo:
          return memo[state_key]

      if state.terminal or depth >= max_depth:
          result = score(state, depth)
          memo[state_key] = result
          return result

      max_value = -float("inf")
      for action in actions:
          new_state = do_action(action, deepcopy(state))
          action_value = maximax_helper(new_state, depth + 1, max_depth)
          max_value = max(max_value, action_value)

      memo[state_key] = max_value
      return max_value
  for action in actions:
        new_state = do_action(action, deepcopy(state))
        action_value = maximax_helper(new_state, 0, max_depth)
        if action_value > best_action_value:
            best_action = action
            best_action_value = action_value
  ### YOUR CODE GOES ABOVE
  return best_action, best_action_value


## Run Maximax

Run the agent using the maximax decision-making algorithm. This runs maximax after every action as if the tree is unreliable after execution, which is theoretically true due to depth limiting.

In [35]:
def run_maximax(state, actions = [act for act in Actions], max_depth = float("inf")):
  # Print the state
  pretty_print(state)
  # Run until terminal state reached
  while not state.terminal:
    # Compute best action
    action, value = maximax(state, actions, max_depth = max_depth)
    print("\nACTION", action.name, "value", value, '\n')
    # Execute the action
    state = do_action(action, state)
    # print the state
    pretty_print(state)


## Test Maximax

Run the following test cases. You will be graded on whether you win each game. Make sure your submit a notebook with cell outputs saved in the notebook so that we can examine your runs. Please disable any debugging print statement before submission so we can easily inspect the game traces.

You may change the ```max_depth``` parameter (though you shouldn't need to). You may NOT change the test case initial states.

**init state 1**




In [36]:
init_state = deepcopy(init_state1)
pretty_print(init_state)

[] t0 [] 
A  [] [] 
[] [] b0 
  guards {'b0': ['t0']}
  agent ALIVE
  towers ['t0: ALIVE']
  bases ['b0: ALIVE']
  terminal False


In [37]:
action, value = maximax(init_state, max_depth=10)
print("action:", action.name)
print("value:", value)

action: EAST
value: 296


In [38]:
run_maximax(init_state, max_depth=10)

[] t0 [] 
A  [] [] 
[] [] b0 
  guards {'b0': ['t0']}
  agent ALIVE
  towers ['t0: ALIVE']
  bases ['b0: ALIVE']
  terminal False

ACTION EAST value 296 

[] t0 [] 
[] A  [] 
[] [] b0 
  guards {'b0': ['t0']}
  agent ALIVE
  towers ['t0: ALIVE']
  bases ['b0: ALIVE']
  terminal False

ACTION NORTH value 297 

[] A  [] 
[] [] [] 
[] [] b0 
  guards {'b0': ['t0']}
  agent ALIVE
  towers ['t0: DEAD']
  bases ['b0: ALIVE']
  terminal False

ACTION EAST value 298 

[] [] A  
[] [] [] 
[] [] b0 
  guards {'b0': ['t0']}
  agent ALIVE
  towers ['t0: DEAD']
  bases ['b0: ALIVE']
  terminal False

ACTION SOUTH value 299 

[] [] [] 
[] [] A  
[] [] b0 
  guards {'b0': ['t0']}
  agent ALIVE
  towers ['t0: DEAD']
  bases ['b0: ALIVE']
  terminal False

ACTION SOUTH value 300 

[] [] [] 
[] [] [] 
[] [] A  
  guards {'b0': ['t0']}
  agent ALIVE
  towers ['t0: DEAD']
  bases ['b0: DEAD']
  terminal True


**Init state 2**

In [39]:
init_state = deepcopy(init_state2)
pretty_print(init_state)

[] t0 [] 
A  [] b0 
[] t1 [] 
  guards {'b0': ['t0', 't1']}
  agent ALIVE
  towers ['t0: ALIVE', 't1: ALIVE']
  bases ['b0: ALIVE']
  terminal False


In [40]:
action, value = maximax(init_state, max_depth=10)
print("action:", action.name)
print("value:", value)

action: EAST
value: 395


In [41]:
run_maximax(init_state, max_depth=10)

[] t0 [] 
A  [] b0 
[] t1 [] 
  guards {'b0': ['t0', 't1']}
  agent ALIVE
  towers ['t0: ALIVE', 't1: ALIVE']
  bases ['b0: ALIVE']
  terminal False

ACTION EAST value 395 

[] t0 [] 
[] A  b0 
[] t1 [] 
  guards {'b0': ['t0', 't1']}
  agent ALIVE
  towers ['t0: ALIVE', 't1: ALIVE']
  bases ['b0: ALIVE']
  terminal False

ACTION SOUTH value 396 

[] t0 [] 
[] [] b0 
[] A  [] 
  guards {'b0': ['t0', 't1']}
  agent ALIVE
  towers ['t0: ALIVE', 't1: DEAD']
  bases ['b0: ALIVE']
  terminal False

ACTION NORTH value 397 

[] t0 [] 
[] A  b0 
[] [] [] 
  guards {'b0': ['t0', 't1']}
  agent ALIVE
  towers ['t0: ALIVE', 't1: DEAD']
  bases ['b0: ALIVE']
  terminal False

ACTION NORTH value 398 

[] A  [] 
[] [] b0 
[] [] [] 
  guards {'b0': ['t0', 't1']}
  agent ALIVE
  towers ['t0: DEAD', 't1: DEAD']
  bases ['b0: ALIVE']
  terminal False

ACTION EAST value 399 

[] [] A  
[] [] b0 
[] [] [] 
  guards {'b0': ['t0', 't1']}
  agent ALIVE
  towers ['t0: DEAD', 't1: DEAD']
  bases ['b0: ALIVE']
 

**Init state3**

In [42]:
init_state = deepcopy(init_state3)
pretty_print(init_state)

[] [] t0 [] [] 
A  [] [] [] [] 
[] [] [] [] b0 
  guards {'b0': ['t0']}
  agent ALIVE
  towers ['t0: ALIVE']
  bases ['b0: ALIVE']
  terminal False


In [43]:
action, value = maximax(init_state, max_depth=10)
print("action:", action.name)
print("value:", value)

action: EAST
value: 294


In [44]:
run_maximax(init_state, max_depth=10)

[] [] t0 [] [] 
A  [] [] [] [] 
[] [] [] [] b0 
  guards {'b0': ['t0']}
  agent ALIVE
  towers ['t0: ALIVE']
  bases ['b0: ALIVE']
  terminal False

ACTION EAST value 294 

[] [] t0 [] [] 
[] A  [] [] [] 
[] [] [] [] b0 
  guards {'b0': ['t0']}
  agent ALIVE
  towers ['t0: ALIVE']
  bases ['b0: ALIVE']
  terminal False

ACTION EAST value 295 

[] [] t0 [] [] 
[] [] A  [] [] 
[] [] [] [] b0 
  guards {'b0': ['t0']}
  agent ALIVE
  towers ['t0: ALIVE']
  bases ['b0: ALIVE']
  terminal False

ACTION NORTH value 296 

[] [] A  [] [] 
[] [] [] [] [] 
[] [] [] [] b0 
  guards {'b0': ['t0']}
  agent ALIVE
  towers ['t0: DEAD']
  bases ['b0: ALIVE']
  terminal False

ACTION EAST value 297 

[] [] [] A  [] 
[] [] [] [] [] 
[] [] [] [] b0 
  guards {'b0': ['t0']}
  agent ALIVE
  towers ['t0: DEAD']
  bases ['b0: ALIVE']
  terminal False

ACTION EAST value 298 

[] [] [] [] A  
[] [] [] [] [] 
[] [] [] [] b0 
  guards {'b0': ['t0']}
  agent ALIVE
  towers ['t0: DEAD']
  bases ['b0: ALIVE']
  term

**Init state 4**

In [45]:
init_state = deepcopy(init_state4)
pretty_print(init_state)

[] [] T2 T0 B0 
A  [] [] [] [] 
[] B1 T1 T3 [] 
  guards {'B0': ['T0'], 'B1': ['T1'], 'T0': ['T2'], 'T1': ['T3']}
  agent ALIVE
  towers ['T0: ALIVE', 'T1: ALIVE', 'T2: ALIVE', 'T3: ALIVE']
  bases ['B0: ALIVE', 'B1: ALIVE']
  terminal False


In [46]:
action, value = maximax(init_state, max_depth=10)
print("action:", action.name)
print("value:", value)

action: EAST
value: 791


In [47]:
run_maximax(init_state, max_depth=10)

[] [] T2 T0 B0 
A  [] [] [] [] 
[] B1 T1 T3 [] 
  guards {'B0': ['T0'], 'B1': ['T1'], 'T0': ['T2'], 'T1': ['T3']}
  agent ALIVE
  towers ['T0: ALIVE', 'T1: ALIVE', 'T2: ALIVE', 'T3: ALIVE']
  bases ['B0: ALIVE', 'B1: ALIVE']
  terminal False

ACTION EAST value 791 

[] [] T2 T0 B0 
[] A  [] [] [] 
[] B1 T1 T3 [] 
  guards {'B0': ['T0'], 'B1': ['T1'], 'T0': ['T2'], 'T1': ['T3']}
  agent ALIVE
  towers ['T0: ALIVE', 'T1: ALIVE', 'T2: ALIVE', 'T3: ALIVE']
  bases ['B0: ALIVE', 'B1: ALIVE']
  terminal False

ACTION EAST value 792 

[] [] T2 T0 B0 
[] [] A  [] [] 
[] B1 T1 T3 [] 
  guards {'B0': ['T0'], 'B1': ['T1'], 'T0': ['T2'], 'T1': ['T3']}
  agent ALIVE
  towers ['T0: ALIVE', 'T1: ALIVE', 'T2: ALIVE', 'T3: ALIVE']
  bases ['B0: ALIVE', 'B1: ALIVE']
  terminal False

ACTION NORTH value 793 

[] [] A  T0 B0 
[] [] [] [] [] 
[] B1 T1 T3 [] 
  guards {'B0': ['T0'], 'B1': ['T1'], 'T0': ['T2'], 'T1': ['T3']}
  agent ALIVE
  towers ['T0: ALIVE', 'T1: ALIVE', 'T2: DEAD', 'T3: ALIVE']
  bases [

**Init state 5**

In [48]:
init_state = deepcopy(init_state5)
pretty_print(init_state)

[] T3 [] B1 [] 
[] [] T2 [] [] 
A  [] [] [] [] 
[] T1 [] T0 [] 
[] [] T4 [] B0 
  guards {'B0': ['T0'], 'B1': ['T1'], 'T1': ['T2'], 'T0': ['T3', 'T4']}
  agent ALIVE
  towers ['T3: ALIVE', 'T2: ALIVE', 'T1: ALIVE', 'T0: ALIVE', 'T4: ALIVE']
  bases ['B0: ALIVE', 'B1: ALIVE']
  terminal False


In [49]:
action, value = maximax(init_state, max_depth=17)
print("action:", action.name)
print("value:", value)

action: EAST
value: 883


In [50]:
run_maximax(init_state, max_depth=17)

[] T3 [] B1 [] 
[] [] T2 [] [] 
A  [] [] [] [] 
[] T1 [] T0 [] 
[] [] T4 [] B0 
  guards {'B0': ['T0'], 'B1': ['T1'], 'T1': ['T2'], 'T0': ['T3', 'T4']}
  agent ALIVE
  towers ['T3: ALIVE', 'T2: ALIVE', 'T1: ALIVE', 'T0: ALIVE', 'T4: ALIVE']
  bases ['B0: ALIVE', 'B1: ALIVE']
  terminal False

ACTION EAST value 883 

[] T3 [] B1 [] 
[] [] T2 [] [] 
[] A  [] [] [] 
[] T1 [] T0 [] 
[] [] T4 [] B0 
  guards {'B0': ['T0'], 'B1': ['T1'], 'T1': ['T2'], 'T0': ['T3', 'T4']}
  agent ALIVE
  towers ['T3: ALIVE', 'T2: ALIVE', 'T1: ALIVE', 'T0: ALIVE', 'T4: ALIVE']
  bases ['B0: ALIVE', 'B1: ALIVE']
  terminal False

ACTION EAST value 884 

[] T3 [] B1 [] 
[] [] T2 [] [] 
[] [] A  [] [] 
[] T1 [] T0 [] 
[] [] T4 [] B0 
  guards {'B0': ['T0'], 'B1': ['T1'], 'T1': ['T2'], 'T0': ['T3', 'T4']}
  agent ALIVE
  towers ['T3: ALIVE', 'T2: ALIVE', 'T1: ALIVE', 'T0: ALIVE', 'T4: ALIVE']
  bases ['B0: ALIVE', 'B1: ALIVE']
  terminal False

ACTION NORTH value 885 

[] T3 [] B1 [] 
[] [] A  [] [] 
[] [] [] [] [

**Init state 6**

In [51]:
init_state = deepcopy(init_state6)
pretty_print(init_state)

A  [] [] [] [] [] [] 
[] [] [] [] [] [] [] 
[] [] [] [] [] [] [] 
B0 [] [] [] [] [] [] 
[] [] [] [] [] [] [] 
[] [] [] [] [] [] [] 
[] [] [] T0 [] [] T1 
  guards {'B0': ['T0'], 'T0': ['T1']}
  agent ALIVE
  towers ['T0: ALIVE', 'T1: ALIVE']
  bases ['B0: ALIVE']
  terminal False


In [52]:
action, value = maximax(init_state, max_depth=30)
print("action:", action.name)
print("value:", value)

action: EAST
value: 380


In [53]:
run_maximax(init_state, max_depth=30)

A  [] [] [] [] [] [] 
[] [] [] [] [] [] [] 
[] [] [] [] [] [] [] 
B0 [] [] [] [] [] [] 
[] [] [] [] [] [] [] 
[] [] [] [] [] [] [] 
[] [] [] T0 [] [] T1 
  guards {'B0': ['T0'], 'T0': ['T1']}
  agent ALIVE
  towers ['T0: ALIVE', 'T1: ALIVE']
  bases ['B0: ALIVE']
  terminal False

ACTION EAST value 380 

[] A  [] [] [] [] [] 
[] [] [] [] [] [] [] 
[] [] [] [] [] [] [] 
B0 [] [] [] [] [] [] 
[] [] [] [] [] [] [] 
[] [] [] [] [] [] [] 
[] [] [] T0 [] [] T1 
  guards {'B0': ['T0'], 'T0': ['T1']}
  agent ALIVE
  towers ['T0: ALIVE', 'T1: ALIVE']
  bases ['B0: ALIVE']
  terminal False

ACTION EAST value 381 

[] [] A  [] [] [] [] 
[] [] [] [] [] [] [] 
[] [] [] [] [] [] [] 
B0 [] [] [] [] [] [] 
[] [] [] [] [] [] [] 
[] [] [] [] [] [] [] 
[] [] [] T0 [] [] T1 
  guards {'B0': ['T0'], 'T0': ['T1']}
  agent ALIVE
  towers ['T0: ALIVE', 'T1: ALIVE']
  bases ['B0: ALIVE']
  terminal False

ACTION EAST value 382 

[] [] [] A  [] [] [] 
[] [] [] [] [] [] [] 
[] [] [] [] [] [] [] 
B0 [] [] [] [] [